In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# 載入檔案
file_path = 'Primary_Transform_exclude benign.xlsx'
data = pd.read_excel(file_path)

# 數據預處理
data['Group'] = data['Group'].map({'NT': 0, 'T': 1})
X = data.drop('Group', axis=1)
y = data['Group']
X.columns = X.columns.astype(str)

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 應用 MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# 定義隨機森林分類器
rf = RandomForestClassifier(random_state=42, oob_score=True)

# 定義超參數網格
param_grid = {
    'n_estimators': [50, 100, 200, 500, 1000],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [4, 6, 8, 10, 12, 15],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True],
    'criterion': ['gini', 'entropy']
}

# 使用GridSearchCV進行超參數調整
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# 訓練模型
grid_search.fit(X_train_scaled, y_train)
# 獲取最佳參數
best_params = grid_search.best_params_
print("最佳參數: ", best_params)

# 使用最佳參數訓練最終模型
best_rf = RandomForestClassifier(**best_params, random_state=42, oob_score=True)
best_rf.fit(X_train_scaled, y_train)

# 使用OOB樣本評估
print("OOB分數: ", best_rf.oob_score_)

# 預測測試集
y_pred_test = best_rf.predict(X_test_scaled)

# 評估模型
print("測試集分類報告:\n", classification_report(y_test, y_pred_test))
print("測試集準確度: ", accuracy_score(y_test, y_pred_test))


In [ ]:
# 交叉验证结果
print("交叉验证分数: ", grid_search.best_score_)